In [58]:
import polars as pl

In [59]:
cahps_df = pl.read_parquet('../../003_data/002_clean-data/national_cms_dialysis-facility_cahps-data.parquet')
facility_df = pl.read_parquet('../../003_data/002_clean-data/national_cms_dialysis-facility_data.parquet')
ballot_measures_df = pl.read_csv('../../003_data/001_raw-data/Ballot Measures Data Clean.csv')
supp_facility_df = pl.read_parquet('../../003_data/001_raw-data/2013-2023_CHHS_dialysis-facility_data.parquet')


In [60]:
cahps_df

year,provider_number,network,facility_name,address_line_1,address_line_2,city,state,zip_code,county,profit_or_nonprofit,phone_number,chain_owned,chain_organization,ichcahps_date,ichcahps_data_availability_code,linearized_score_of_nephrologists_communication_and_caring,star_rating_of_nephrologists_communication_and_caring,linearized_score_of_quality_of_dialysis_center_care_and_operations,star_rating_of_quality_of_dialysis_center_care_and_operations,linearized_score_of_providing_information_to_patients,star_rating_of_providing_information_to_patients,linearized_score_of_rating_of_the_nephrologist,star_rating_of_the_nephrologist,linearized_score_of_rating_of_the_dialysis_center_staff,star_rating_of_the_dialysis_center_staff,linearized_score_of_rating_of_the_dialysis_facility,star_rating_of_the_dialysis_facility,total_number_of_completed_interviews_from_the_fall_and_spring_surveys,ich_cahps_survey_of_patients_experiences_star_rating,survey_response_rate,ichcahps_survey_response_rate,ich_cahps_quality_of_patient_care_star_rating
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2024""","""52305""","""17""","""SANTA CLARA VALLEY RENAL CARE …","""2220 MOORPARK AVENUE""",null,"""SAN JOSE""","""CA""","""95128""","""Santa Clara""","""non-profit""","""(408) 885-5730""","""no""","""Independent""","""21OCT2022-14JUL2023""","""1""","""85""","""4""","""85""","""5""","""81""","""4""","""88""","""5""","""88""","""4""","""89""","""3""","""47""","""4""","""19""",null,null
"""2024""","""52311""","""18""","""St. Joseph Hospital Renal Cent…","""Sr. Elizabeth Bldg. 1100 W. St…",null,"""Orange""","""CA""","""92868""","""Orange""","""non-profit""","""(714) 771-8037""","""no""","""Independent""","""21OCT2022-14JUL2023""","""101""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024""","""52321""","""18""","""Childrens Hospital of Los Ange…","""Division of Nephrology (Dialys…",null,"""Los Angeles""","""CA""","""90027""","""Los Angeles""","""non-profit""","""(323) 361-2560""","""no""","""Independent""","""21OCT2022-14JUL2023""","""102""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2024""","""52323""","""18""","""Kaiser Foundation Hospital Med…","""Dialysis Unit 4700 Sunset Blvd…",null,"""Los Angeles""","""CA""","""90027""","""Los Angeles""","""non-profit""","""(323) 783-5580""","""yes""","""Kaiser Permanente""","""21OCT2022-14JUL2023""","""1""","""76""","""2""","""82""","""4""","""84""","""5""","""82""","""3""","""88""","""4""","""91""","""4""","""52""","""4""","""25""",null,null
"""2024""","""52334""","""18""","""Arrowhead Regional Medical Cen…","""400 N. Pepper Avenue""",null,"""Colton""","""CA""","""92324""","""San Bernardino""","""non-profit""","""(909) 580-3911""","""no""","""Independent""","""21OCT2022-14JUL2023""","""101""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2018""","""552833""","""18""","""U.S. RENAL CARE SOUTH BAY DIAL…","""4450 182ND STREET""","""""","""REDONDO BEACH""","""CA""","""90278""","""LOS ANGELES""","""profit""","""(310) 542-8032""","""yes""","""US RENAL CARE, INC.""","""04/21/2017 - 01/17/2018""","""258""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""",null,null,null,"""not Available"""
"""2018""","""552834""","""18""","""DAVITA-VISTA DEL SOL DIALYSIS""","""15002 AMARGOSA ROAD""","""""","""VICTORVILLE""","""CA""","""92394""","""SAN BERNARDINO""","""profit""","""(442) 255-4023""","""yes""","""DAVITA""","""04/21/2017 - 01/17/2018""","""258""","""not Available""","""not Available""","""not Available""","""not Available""","""not Available""","""

In [61]:
ballot_measures_df = ballot_measures_df.rename({"county name": "county"})

ballot_measures_df = (
    ballot_measures_df
    .unpivot(
        index = ['county'],
        on = ['yes count 2022', 'no count 2022' ,'yes count 2020', 'no count 2020', 'yes count 2018', 'no count 2018'],
        value_name = "vote_count"
    )
    .with_columns(
    pl.col("variable")
    .str.split_exact(" count ", 1)
    .struct.rename_fields(["vote_type", "year"])
    .alias("fields")
    ).unnest("fields")
    .drop("variable")
)



In [62]:
ballot_measures_df = (
    ballot_measures_df
    .with_columns(
        pl.col("year")
        .str.to_date("%Y")
    )
)

ballot_measures_df

county,vote_count,vote_type,year
str,f64,str,date
"""Alameda""",182697.0,"""yes""",2022-01-01
"""Alpine""",272.0,"""yes""",2022-01-01
"""Amador""",3590.0,"""yes""",2022-01-01
"""Butte""",16478.0,"""yes""",2022-01-01
"""Calaveras""",4280.0,"""yes""",2022-01-01
…,…,…,…
"""Tulare""",69542.0,"""no""",2018-01-01
"""Tuolumne""",15546.0,"""no""",2018-01-01
"""Ventura""",187357.0,"""no""",2018-01-01


In [63]:
facility_df.head()

provider_number,network,facility_name,five_star_date,five_star,five_star_data_availability_code,address_line_1,address_line_2,state,zip_code,profit_or_nonprofit,chain_owned,chain_organization,late_shift,_of_dialysis_stations,offers_incenter_hemodialysis,offers_peritoneal_dialysis,offers_home_hemodialysis_training,certification_date,claims_date,eqrs_date,smr_date,patient_survival_category_text,patient_survival_data_availability_code,number_of_patients_included_in_survival_summary,mortality_rate_facility,mortality_rate_upper_confidence_limit_975,mortality_rate_lower_confidence_limit_25,shr_date,patient_hospitalization_category_text,patient_hospitalization_data_availability_code,number_of_patients_included_in_hospitalization_summary,hospitalization_rate_upper_confidence_limit_975,hospitalization_rate_lower_confidence_limit_25,srr_date,patient_hospital_readmission_category,patient_hospital_readmission_data_availability_code,…,percentage_of_adult_patients_with_serum_phosphorus_between_5670_mgdl,percentage_of_adult_patients_with_serum_phosphorus_greater_than_70_mgdl,long_term_catheter_data_availability_code,number_of_patients_in_long_term_catheter_summary,number_of_patient_months_in_long_term_catheter_summary,percentage_of_adult_patients_with_long_term_catheter_in_use,npcr_data_availability_code,number_of_patients_in_npcr_summary,number_of_patientmonths_in_npcr_summary,percentage_of_pediatric_hd_patients_with_npcr,year,month,city,county,phone_number,certification_or_recertification_date,crownweb_date,serum_phosphorus_data_availability_code_,dateswr,offers_incenter_peritoneal_dialysis,rate_of_hospital_readmission_category_text,percentage_of_medicare_patients_with_hgb_10_gdl,number_of_patients_included_in_transfusion_summary,standard_infection_ratio_,percentage_of_adult_hd_patients_with_ktv_12,percentage_of_adult_pd_patients_with_ktv17,percentage_of_pediatric_hd_patents_with_ktv12,percentage_of_pediatric_pd_patents_with_ktv18,number_of_adult_patients_included_in_arterial_venous_fistula_and_catheter_summaries,number_of_adult_patientmonths_included_in_arterial_venous_fistula_and_catheter_summaries,arteriovenous_fistulae_in_use_data_availability_code,percentage_of_patients_with_arteriovenous_fistulae_in_use,vascular_catheter_data_availability_code,percentage_of_patients_with_vascular_catheter_in_use_for_90_days_or_longer,hospitalization_rate_facility_,patient_hospital_readmission_category_text,readmission_rate_facility_
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""52305""","""17""","""SANTA CLARA VALLEY RENAL CARE …","""01Jan2019-31Dec2022""","""3""","""1""","""2220 MOORPARK AVENUE""",null,"""CA""","""95128""","""non-profit""","""no""","""Independent""","""yes""","""25""","""yes""","""yes""","""no""","""22AUG1977""","""01OCT2022-30SEP2023""","""01OCT2022-30SEP2023""","""01Jan2019-31Dec2022""","""As Expected""","""1""","""529""","""19.8""","""30.5""","""13.3""","""01Jan2022-31Dec2022""","""As Expected""","""1""","""117""","""205.7""","""97.9""","""01Jan2022-31Dec2022""","""As Expected""","""1""",…,"""31""","""11""","""1""","""192""","""1771""","""22""","""259""","""0""",null,null,"""2024""","""7""","""SAN JOSE""","""Santa Clara""","""(408) 885-5730""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""52311""","""18""","""St. Joseph Hospital Renal Cent…","""01Jan2019-31Dec2022""","""3""","""1""","""Sr. Elizabeth Bldg. 1100 W. St…",null,"""CA""","""92868""","""non-profit""","""no""","""Independent""","""yes""","""39""","""yes""","""yes""","""yes""","""15AUG1977""","""01OCT2022-30SEP2023""","""01OCT2022-30SEP2023""","""01Jan2019-31Dec2022""","""As Expected""","""1""","""533""","""22.9""","""33.7""","""16.1""","""01Jan2022-31Dec2

In [64]:

# It seems like dialysis facilites aren't located in more rural parts of california


# Lowercasing all county names in the dataframes

facility_df = (
    facility_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county")
    )
)

ballot_measures_df = (
    ballot_measures_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county")
    )
)

cahps_df = (
    cahps_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county")
    )
)

# Getting unique counties from all dataframes
ballot_counties = set(ballot_measures_df['county'].unique())
facility_counties = set(facility_df['county'].unique())
cahps_counties = set(cahps_df['county'].unique())

print("Unique counties in ballot_measures_df:")
print(sorted(ballot_counties))

print("\nUnique counties in facility_df:")
print(sorted(facility_counties))

print("\nUnique counties in cahps_df:")
print(sorted(cahps_counties))

# Finding counties that are in one dataframe but not others
counties_only_in_ballot = ballot_counties - facility_counties - cahps_counties
counties_only_in_facility = facility_counties - ballot_counties - cahps_counties
counties_only_in_cahps = cahps_counties - ballot_counties - facility_counties

print("\nCounties only in ballot_measures_df:")
print(sorted(counties_only_in_ballot))

print("\nCounties only in facility_df:")
print(sorted(counties_only_in_facility))

print("\nCounties only in cahps_df:")
print(sorted(counties_only_in_cahps))

# Checking for potential mismatches due to formatting
all_counties = ballot_counties.union(facility_counties).union(cahps_counties)
potential_mismatches = [county for county in all_counties if any(
    county != other_county and county in other_county
    for other_county in all_counties
)]

print("\nPotential county name mismatches:")
print(sorted(potential_mismatches))


Unique counties in ballot_measures_df:
['alameda', 'alpine', 'amador', 'butte', 'calaveras', 'colusa', 'contra costa', 'del norte', 'el dorado', 'fresno', 'glenn', 'humboldt', 'imperial', 'inyo', 'kern', 'kings', 'lake', 'lassen', 'los angeles', 'madera', 'marin', 'mariposa', 'mendocino', 'merced', 'modoc', 'mono', 'monterey', 'napa', 'nevada', 'orange', 'placer', 'plumas', 'riverside', 'sacramento', 'san benito', 'san bernardino', 'san diego', 'san francisco', 'san joaquin', 'san luis obispo', 'san mateo', 'santa barbara', 'santa clara', 'santa cruz', 'shasta', 'sierra', 'siskiyou', 'solano', 'sonoma', 'stanislaus', 'sutter', 'tehama', 'trinity', 'tulare', 'tuolumne', 'ventura', 'yolo', 'yuba']

Unique counties in facility_df:
['', 'alameda', 'butte', 'columbia', 'colusa', 'contra costa', 'del norte', 'el dorado', 'fresno', 'humboldt', 'imperial', 'inyo', 'kern', 'kings', 'lake', 'los angeles', 'madera', 'marin', 'mendocino', 'merced', 'monterey', 'napa', 'nevada', 'orange', 'placer',

In [65]:
supp_facility_df

FAC_NO,FAC_NAME,FAC_CITY,FAC_ZIP,FAC_PHONE,FAC_ADMIN_NAME,FAC_OPERATED_THIS_YR,FAC_OP_PER_BEGIN_DT,FAC_OP_PER_END_DT,FAC_PAR_CORP_NAME,FAC_PAR_CORP_CITY,FAC_PAR_CORP_STATE,FAC_PAR_CORP_ZIP,REPT_PREP_NAME,LICENSE_STATUS,LIC_STATUS_DATE,LIC_ORIG_DATE,REPORT_STATUS,MCAL_PROVIDER_NO,MCARE_PROVIDER_NO,ACLAIMS_NO,ASSEMBLY_DIST,SENATE_DIST,CONGRESS_DIST,CENS_TRACT,MED_SVC_STUDY_AREA,LA_COUNTY_SVC_PLAN_AREA,HEALTH_SVC_AREA,COUNTY,LICENSE_NO,LONGITUDE,LATITUDE,LIC_CAT,LICEE_TOC,TOT_UNDUPLICATED_PATIENTS,TOT_ENCOUNTERS,SURG_OPER_RMS_ON_DEC_31_CNT,…,EQUIP_VAL_10,DT_ACQUIRE_10,MEANS_FOR_ACQUISITION_10,PROJ_OVER_1M,DEPROJ_01,PROJ_EXPENDITURES_01,OSHPD_PROJ_NO_01,DEPROJ_02,PROJ_EXPENDITURES_02,OSHPD_PROJ_NO_02,DEPROJ_03,PROJ_EXPENDITURES_03,OSHPD_PROJ_NO_03,DEPROJ_04,PROJ_EXPENDITURES_04,OSHPD_PROJ_NO_04,DEPROJ_05,PROJ_EXPENDITURES_05,OSHPD_PROJ_NO_05,source_file,year,FAC_STR_ADDR,FAC_PAR_CORP_BUS_ADDR,Description,SUBMITTED_DT,REV_REPT_PREP_NAME,REVISED_DT,CORRECTED_DT,LICENSE_EFF_DATE,LICENSE_EXP_DATE,FACILITY_LEVEL,HCAI_PROJ_NO_01,HCAI_PROJ_NO_02,HCAI_PROJ_NO_03,HCAI_PROJ_NO_04,HCAI_PROJ_NO_05,__index_level_0__
f64,str,str,str,str,str,str,datetime[ns],datetime[ns],str,str,str,str,str,str,datetime[ns],datetime[ns],str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,str,str,f64,f64,f64,…,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,f64,datetime[ns],str,str,str,datetime[ns],datetime[ns],str,f64,f64,f64,f64,f64,i64
3.06540615e8,"""ARA KAWAEH DIALYSIS CENTER""","""VISALIA""","""93277""","""5597419263""","""Teresa Shaw""","""Yes""",2023-01-01 00:00:00,2023-12-31 00:00:00,"""Teresa Shaw""","""Visalia""","""CA - California""","""93277""","""Teresa Shaw""","""Open""",null,null,"""""","""""","""""","""""",32.0,12.0,20.0,"""06107001902""","""233""","""""","""09 - Central""","""Tulare""",5.50006398e8,-119.311749,36.298925,"""Chronic Dialysis Clinic""","""Investor - Limited Liability C…",102.0,1164.0,null,…,null,null,null,"""No""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""file_1""","""23""","""3446 S. MOONEY BLVD""","""3446 S Mooney Blvd""",null,2024-02-12 14:29:00,"""""","""""","""""",2023-06-17 00:00:00,2024-06-16 00:00:00,"""Parent Facility""",null,null,null,null,null,0
3.06197943e8,"""HIGH DESERT HEALTH SYSTEM AMBU…","""LANCASTER""","""93535""","""661 471-4000""","""Lee Dunham""","""Yes""",2023-01-01 00:00:00,2023-12-31 00:00:00,"""""","""""","""""","""""","""LINDA KEITH""","""Open""",null,null,"""""","""""","""""","""""",39.0,23.0,27.0,"""06037900602""","""77.1c""","""Antelope Valley""","""11 - Los Angeles""","""Los Angeles""",6.0000872e7,-118.124835,34.704507,"""Surgical Clinic""","""City or County""",1599.0,1599.0,3.0,…,null,null,null,"""No""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""file_1""","""23""","""335 EAST AVENUE I""","""""",null,2024-02-13 14:26:00,"""""","""""","""""",2023-11-01 00:00:00,2024-10-31 00:00:00,"""Parent Facility""",null,null,null,null,null,1
3.06198064e8,"""MARTIN LUTHER KING, JR. AMBULA…","""LOS ANGELES""","""90059""","""213 699-7102""","""Michael mills""","""Yes""",2023-01-01 00:00:00,2023-12-31 00:00:00,"""""","""""","""""","""""","""LINDA KEITH""","""Open""",null,null,"""""","""""","""""","""""",65.0,35.0,43.0,"""06037540700""","""78.2ooo""","""South""","""11 - Los Angeles""","""Los Angeles""",5.50000727e8,-118.243764,33.923232,"""Surgical Clinic""","""City or County""",3834.0,3834.0,5.0,…,null,null,null,"""No""","""""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""file_1""","""23""","""1670 E 120TH ST""","""""",null,2024-02-13 14:56:00,"""""","""""","""""",2023-05-28 00:00:00,2024-05-27 00:00:00,"""Parent Facility""",null,null,null,null,null,2
3.06197915e8,"""SHRINERS FOR CHILDREN AMBULATO…","""PASADENA""","""91105""","""626-389-9300""","""Kanayo Keri""","""Yes""",2023-01-01 00:00:00,2023-12-31 00:00:00,"""Shriners Hospitals for Childre…","""Tampa""","""FL - F